# VQE vs AdaptVQE in Li-H ground state search

In [1]:
#install and load pylint eu
!pip install pycodestyle py
%load_ext pycodestyle_magic

# uncomment %%pycodestyle in the following cell to evaluate coding conventions

In [1]:
#%%pycodestyle
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.algorithms import VQEUCCFactory,GroundStateEigensolver,AdaptVQE

from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SLSQP,L_BFGS_B,CG
from qiskit.utils import QuantumInstance
from qiskit import Aer


from qiskit_mod.qiskit_nat import VHA

import numpy as np 
import time
import math
import matplotlib.pyplot as plt

#CHECK VERSIONS OF QISKIT: 'qiskit-terra': '0.19.0','qiskit-aer': '0.8.2','qiskit-nature': '0.2.0'
import qiskit
print(qiskit.__qiskit_version__)

/home/gsilvi/miniforge3/envs/NEASQC4/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


{'qiskit-terra': '0.19.0', 'qiskit-aer': '0.8.2', 'qiskit-ignis': None, 'qiskit-ibmq-provider': None, 'qiskit-aqua': None, 'qiskit': None, 'qiskit-nature': '0.2.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}


In [3]:
#%%pycodestyle

distances = np.arange(1, 5, 1)  # angstrom distance


# Initialize qubit converter
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)  # two qubit reduction doesn`t work with Jordan-WignerMapper()

# Initialize Exact solvers & VQE solver
numpy_solver = NumPyMinimumEigensolver()


usenoise=False

#NOISE
from qiskit.test.mock import FakeVigo,FakeCasablanca
from qiskit.providers.aer import AerSimulator

if usenoise:
    device_backend = FakeCasablanca()
    simulator = AerSimulator.from_backend(device_backend)
    quantum_instance = QuantumInstance(backend = simulator)
else:
    quantum_instance = QuantumInstance(backend = Aer.get_backend('qasm_simulator'))
    #quantum_instance = QuantumInstance(backend= Aer.get_backend('statevector_simulator'))


optimizer = L_BFGS_B(maxiter=1000)

In [4]:
#%%pycodestyle

exact_energies, hf_energies, vqe_energies, adapt_vqe_energies, vqe_params, adapt_vqe_params, vha_energies, vha_params = ([] for i in range(8))
trotter_steps = 3
time_step_size = 1

# prepare intial point for VHA, inspired by adiabatic evolution
ip = []
for i in range(trotter_steps):
    ip.append(time_step_size*((i+1)/trotter_steps))  # param for H_ex = V
    ip.append(time_step_size)                        # param for H_hop = H_0
    ip.append(time_step_size)                        # param for H_diag = H_0
print('Initial parameter for VHA, '+str(trotter_steps)+'*(H_ex,H_op,H_diag)=', ip)

for i, bond_distance in enumerate(distances):  # loop over distances
    print('\n step', i, ', bond_distance=', bond_distance)
    molecule = 'H .0 .0 -{0}; Li .0 .0 {0}'
    driver = PySCFDriver(atom=molecule.format(bond_distance/2), unit=UnitsType.ANGSTROM, basis='sto3g')
    qmolecule = driver.run()
    es_problem = ElectronicStructureProblem(driver,q_molecule_transformers=[FreezeCoreTransformer(freeze_core=True, remove_orbitals=[4,5])])
    vqe_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer)

    # SOLVERS
    # vqe solver
    calc = GroundStateEigensolver(qubit_converter, vqe_solver)
    # adapt vqe solver
    adapt_calc = AdaptVQE(qubit_converter, vqe_solver, threshold=0.001)
    # exact numpy solver
    exact_calc = GroundStateEigensolver(qubit_converter, numpy_solver)

    # vha solver
    vha_solver = VQEUCCFactory(quantum_instance, optimizer=optimizer, initial_point=np.asarray(ip), ansatz=VHA(excitations='sd', trotter_steps=trotter_steps, only_excitations=True))
    calcVHA = GroundStateEigensolver(qubit_converter, vha_solver)

    # Run the computation

    # compute ground state with VHA
    start = time.time()
    resVHA = calcVHA.solve(es_problem)
    vha_energies.append(resVHA.total_energies)
    vha_params.append(len(resVHA.raw_result.optimal_parameters))
    end = time.time()
    print('VHA ,', trotter_steps, 'steps:',
          resVHA.total_energies, ',exec_time=',
          end - start, 's, number of optimal parameters=',
          len(resVHA.raw_result.optimal_parameters))

    # compute ground state with VQE
    start = time.time()
    res = calc.solve(es_problem)
    vqe_energies.append(res.total_energies)
    hf_energies.append(res.hartree_fock_energy)
    vqe_params.append(len(res.raw_result.optimal_parameters))
    end = time.time()
    print('VQE:\t', res.total_energies,
          ',exec_time=', end - start,
          's, number of optimal parameters=', len(res.raw_result.optimal_parameters))

    # compute ground state with AdaptVQE
    start = time.time()
    res2 = adapt_calc.solve(es_problem)
    adapt_vqe_energies.append(res2.total_energies)
    adapt_vqe_params.append(len(res2.raw_result.optimal_parameters))
    end = time.time()
    print('AdaptVQE:', res2.total_energies,
          ',exec_time=', end - start,
          's, number of iterations', res2._num_iterations,
          ',number of optimal parameters=', len(res2.raw_result.optimal_parameters),
          '\n(final max gradient', res2._final_max_gradient,
          'final criterion', res2._finishing_criterion, ')')

    # compute exact ground state
    start = time.time()
    res3 = exact_calc.solve(es_problem)
    exact_energies.append(res3.total_energies)
    end = time.time()
    print('Exact:\t', res3.total_energies, ',exec_time=', end - start, 's')

Initial parameter for VHA, 3*(H_ex,H_op,H_diag)= [0.3333333333333333, 1, 1, 0.6666666666666666, 1, 1, 1.0, 1, 1]

 step 0 , bond_distance= 1


/home/gsilvi/Projects/NEASQ-4/Variationals_algorithms/misc/notebooks/qiskit_mod/qiskit_nat/ansatzVHA.py:156: DeprecationWarning: The EvolvedOperatorAnsatz class is deprecated as of version 0.2.0 and will be removed no sooner than 3 months after the release. Instead use the EvolvedOperatorAnsatz class from qiskit.circuit.library as a direct replacement.
  super().__init__(reps=reps, evolution=PauliTrotterEvolution(), initial_state=initial_state)
The measured number of particles 3.007812499999999 does NOT match the expected number of particles 2!


VHA , 3 steps: [-7.19569996+0.j] ,exec_time= 318.0616628909993 s, number of optimal parameters= 9
VQE:	 [-7.76658615+0.j] ,exec_time= 41.43665115999829 s, number of optimal parameters= 8
AdaptVQE: [-7.7662108+0.j] ,exec_time= 49.74517491599909 s, number of iterations 2 ,number of optimal parameters= 1 
(final max gradient 0.01597163968166687 final criterion Aborted due to cyclicity )
Exact:	 [-7.76862177+0.j] ,exec_time= 2.7165099150006426 s

 step 1 , bond_distance= 2


KeyboardInterrupt: 

In [ ]:
#%%pycodestyle

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))

ax1.plot(distances, hf_energies, color='blue', label='Hartree-Fock')
ax1.plot(distances, vqe_energies, 'o', color='orange', label='VQE')
ax1.plot(distances, adapt_vqe_energies, 'x', color='green', label='AdaptVQE (threshold=0.001)')
ax1.plot(distances, vha_energies, '.', color='red', label='VHA ('+str(trotter_steps)+' steps)')
ax1.plot(distances, exact_energies, '.', color='black', label='Exact')
ax1.set_xlabel('Interatomic distance')
ax1.set_ylabel('Energy')
ax1.set_title('H-Li Ground State Energy')
ax1.legend(loc='upper left')


ypoints = [abs(a - b) for a, b in zip(hf_energies, exact_energies)]
ax2.plot(distances, ypoints, color='blue', label='Hartree-Fock')
ypoints = [abs(a - b) for a, b in zip(vqe_energies, exact_energies)]
ax2.plot(distances, ypoints, 'o', linestyle='--', color='orange', label='VQE')
ypoints = [abs(a - b) for a, b in zip(adapt_vqe_energies, exact_energies)]
ax2.plot(distances, ypoints, 'x', linestyle='--', color='green', label='AdaptVQE')
ypoints = [abs(a - b) for a, b in zip(vha_energies, exact_energies)]
ax2.plot(distances, ypoints, '.', linestyle='--', color='red', label='VHA('+str(trotter_steps)+' steps)')


ax2.set_yscale('log')
ax2.set_xlabel('Interatomic distance')
ax2.set_ylabel('|E-E_{exact}|')
ax2.set_title('H-Li Ground State Energy error')
ax2.legend(loc='upper left')


ax3.plot(distances, vqe_params, 'o', linestyle='-.', color='orange', label='VQE')
ax3.plot(distances, adapt_vqe_params, 'x', linestyle='-.', color='green', label='AdaptVQE')
ax3.plot(distances, vha_params, '.', linestyle='-.', color='red', label='VHA('+str(trotter_steps)+' steps)')

ax3.set_xlabel('Interatomic distance')
ax3.set_ylabel('Number of parameters')
ax3.set_title('Number of optimal parameters')
ax3.legend(loc='upper left')


fig.savefig('LiH_ground_energy2big_w_noise.png')